In [1]:
import pathlib
import numpy as np

import tensorflow as tf 
from tensorflow.keras import layers
import h5py
import re

2024-04-28 11:08:53.500052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
imported = tf.saved_model.load('saved')
m = imported.model

In [3]:
m.keras_api

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x12f82ff50>

In [4]:
h5_model = tf.keras.models.load_model('simple-sr.h5')
h5_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 normalization (Normalizatio  (None, 32, 32, 1)        3         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0

In [5]:
def make_var_name(v):
    s = re.split('/|:', v.name)[:-1]
    return 'weights_' + '_'.join(s)

def make_type(v):
    return re.split('[0-9]', v.dtype.name)[0]

def c_write(f, s=''):
    #print(s, end='')
    f.write(s)

def create_var_prim(f, name, typ, val, extern=False):
    if extern:
        c_write(f, f'extern {typ} {name};\n\n')
        return

    c_write(f, f'{typ} {name} = {val};\n\n')
    
def create_var_arr(f, name, typ, arr, shape, extern=False):
    if extern:
        dims = [1,1,1,1]
        if len(shape) == 1:
            dims[3] = shape[0]
        elif len(shape) == 2:
            dims[2] = shape[0]
            dims[3] = shape[1]
        elif len(shape) == 3:
            dims[1] = shape[0]
            dims[2] = shape[1]
            dims[3] = shape[2]
        else:
            dims[0] = shape[0]
            dims[1] = shape[1]
            dims[2] = shape[2]
            dims[3] = shape[3]

        for i in range(len(dims)):
            c_write(f, f'const int {name}_d{i} = {dims[i]};\n')

        len_name = f'{name}_len'
        c_write(f, f'const int {len_name} = {len(arr)};\n')
        c_write(f, f'extern {typ} {name}[{len_name}];\n\n')
        return

    c_write(f, '%s %s[] = {\n    ' % (typ, name))

    data = v.numpy().flatten()

    for i in range(len(data)):
        if i != 0 and i % 16 == 0:
            c_write(f, '\n    ')
        if i == len(data) - 1:
            c_write(f, f'{data[i]}')
        else:
            c_write(f, f'{data[i]}, ')
    c_write(f, '\n};\n\n')

def create_var(f, v, extern=False):
    c_write(f, '// dimensions: %s\n' % str(v.shape))

    name = make_var_name(v)
    typ  = make_type(v)
    arr  = v.numpy().flatten()
    if len(arr) > 1:
        create_var_arr(f, name, typ, arr, v.numpy().shape, extern)
    else:
        create_var_prim(f, name, typ, arr[0], extern)

In [6]:
with open('weights.h', 'w') as f:
    c_write(f, '#ifndef __WEIGHTS_H__\n')
    c_write(f, '#define __WEIGHTS_H__\n')
    
    for v in m.variables:
        create_var(f, v, extern=True)

    c_write(f, '#endif  // __WEIGHTS_H__\n')

with open('weights.cc', 'w') as f:
    c_write(f, '#include "weights.h"\n\n')
    
    for v in m.variables:
        create_var(f, v)

In [7]:
for v in m.variables:
    print(v.name, v.shape)

mean:0 (1,)
variance:0 (1,)
count:0 ()
conv2d/kernel:0 (3, 3, 1, 32)
conv2d/bias:0 (32,)
conv2d_1/kernel:0 (3, 3, 32, 64)
conv2d_1/bias:0 (64,)
dense/kernel:0 (12544, 128)
dense/bias:0 (128,)
dense_1/kernel:0 (128, 8)
dense_1/bias:0 (8,)
